In [6]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from the .env file
load_dotenv()

# Check if the OPENAI_API_KEY is in the environment variables
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your groq API Key: ")

# Now, you can use the API key as needed
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")


In [7]:
print(GROQ_API_KEY)

In [ ]:
import os
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load pre-trained model and tokenizer
model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to generate a response given a prompt and conversation history
def generate_response(prompts, history):
    # Preprocess the prompts and history
    inputs = []
    for prompt in prompts:
        inputs.append(tokenizer.encode_plus(
            prompt,
            history,
            add_special_tokens=True,
            max_length=512,
            return_attention_mask=True,
            return_tensors="pt",
        ))

    # Generate responses
    outputs = model.generate(
        torch.cat([input["input_ids"] for input in inputs]),
        attention_mask=torch.cat([input["attention_mask"] for input in inputs]),
        max_length=512,
    )

    # Convert responses to text
    responses = []
    for output in outputs:
        responses.append(tokenizer.decode(output, skip_special_tokens=True))

    return responses

# Define a function to manage the conversation history
def manage_history(history, new_messages):
    # Append the new messages to the history
    history.extend(new_messages)

    # Return the updated history
    return history

# Initialize the conversation history
history = []

# Test the chatbot
while True:
    # Get user input
    user_inputs = input("User: ").split(",")

    # Add the user inputs to the history
    history = manage_history(history, user_inputs)

    # Generate responses
    responses = generate_response(user_inputs, history)

    # Print the responses
    for response in responses:
        print("AI:", response)